# 1. Preparation
## 1.1. Loading Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
from typing import Callable, List


np.set_printoptions(precision=4)

## 1.2. Numerical Simulation of Ordinary Differential Equations

Implemented the classic RK4 method and the explicit first-order Euler method.

In [2]:
def runge_kutta_4(f: Callable, x: np.ndarray, dt: float):
    '''
    Rk4 Simulator for ordinary differential equations.
    Given dx/dt = f(x) and x(t), compute x(t+dt)
    '''

    k1 = f(x)
    k2 = f(x + dt/2 * k1)
    k3 = f(x + dt/2 * k2)
    k4 = f(x + dt * k3)
    s = (k1 + 2*k2 + 2*k3 + k4)/6
    return x + s*dt


def euler(f: Callable, x: np.ndarray, dt: float):
    '''
    Euler Simulator for ordinary differential equations.
    Given dx/dt = f(x) and x(t), compute x(t+dt)
    '''

    s = f(x)
    return x + s*dt

## 1.3. Differentiable Function Class

Unifying function evaluation and differentiation into a single object makes the logic of subsequent code clearer.

In [3]:
class Differentiable:
    '''
    A class for differentiable funcitons.
    '''

    def __init__(self, f: Callable, df: Callable):
        self.__f = f
        self.__df = df

    def __call__(self, x, d=False):
        if d:
            return self.__df(x)
        else:
            return self.__f(x)

# 2. Class Implementation for Solving General Optimization Problems

Define a class that describes an optimization problem and solves it using a method similar to the Hopfield-NN approach.

In [4]:
class Minimization:

    # Penalty function
    penalty = Differentiable(lambda x: max(x, 0)**2,
                             lambda x: 2*x if x >= 0 else 0)

    def __init__(self, f: Differentiable, gs: List[Differentiable],
                 tau=1.0, penalty_rate=1.0):
        '''
        Assume x is an n-dim variable.
        Minimize `f(x)` subject to `g(x)` <= 0, for all `g` in `gs`.

        args:
        - `f`: the target function to minimize
        - `gs`: a list of constrains, where g(x) <= 0 for all g in gs.
        - `tau`: time constant controlling the speed of the dynamics.
        - `penalty_rate`: the constant controlling the magnitude of the
                          penalty function.
        '''

        self.f = f
        self.gs = gs
        self.penalty_rate = penalty_rate
        self.tau = tau

    def set_penalty_rate(self, value):
        self.penalty_rate = value

    def sum_penalty(self, x):
        return sum(self.penalty(g(x)) for g in self.gs)

    def energy(self, x):
        '''
        The energy function E(x)
        '''
        return self.f(x) + self.penalty_rate * self.sum_penalty(x)

    def dynamics(self, x):
        '''
        dx/dt where x(t) is given
        '''

        gs = np.array([g(x) for g in self.gs])
        dgdx = np.array([g(x, d=True) for g in self.gs])
        dpdg = np.array([self.penalty(g, d=True) for g in gs])

        return - self.tau * (self.f(x, d=True) +
                             self.penalty_rate * dpdg @ dgdx)

    def iterate(self, x0, dt=1e-4, T=0.2, simulator=runge_kutta_4, pr=None):
        '''
        Simulate the dynamics of the network from `x0` and carry `nstep`
        iterations, among which the interval is set to `dt`. We can choose
        a specific (or costomed) ODE simulator, like Euler or Runge Kutta
        method.
        '''
        if pr is not None:
            self.set_penalty_rate(pr)

        x = x0
        t = 0
        k = 0
        while t < T:
            p1 = [self.penalty(g(x)) for g in self.gs][0]
            p2 = [self.penalty(g(x)) for g in self.gs][1]
            p3 = [self.penalty(g(x)) for g in self.gs][2]
            p4 = [self.penalty(g(x)) for g in self.gs][3]
            p = self.sum_penalty(x)
            e = self.energy(x)
            xnext = simulator(self.dynamics, x, dt)
            yield t, x, e, p, p1,p2,p3,p4, xnext

            k += 1
            t = k * dt
            x = xnext

# 3. Problem Solving
## 3.1 Problem Definition

In [5]:
f = Differentiable(
    lambda x: (x[0]*0.5 + 0.5*(5*x[0]**2+8*x[1]**2+4*x[2]**2)-4*x[0]*x[1]-4*x[1]*x[2]),
    lambda x: np.array([0.5+5*x[0]-4*x[1],
                       8*x[1]-4*x[0]-4*x[2],
                       4*x[2]-4*x[1]])
)


g1 = Differentiable(
    lambda x: 1-x[0]-x[1]-x[2],
    lambda x: np.array([-1, -1, -1])
)

g2 = Differentiable(
    lambda x: -x[0],
    lambda x: np.array([-1,0,0]
                       ))

g3 = Differentiable(
    lambda x: -x[1],
    lambda x: np.array([0,-1,0])
)

g4 = Differentiable(
    lambda x: -x[2],
    lambda x: np.array([0,0,-1])
)

m = Minimization(f, [g1, g2, g3, g4], penalty_rate=2000.0)

## 3.2 Solution



In [6]:
x0 = np.random.rand(3)
# x0 = np.array([0.4, 0.4,0.4])


def solve(export_to: str, *args, **kargs):
    df = pd.DataFrame(
        columns=["t", "x0", "x1", "energy", "penalty", "p1", "p2", "p3", "p4"]
    )

    for k, r in enumerate(m.iterate(*args, **kargs)):
        t, x, e, p, p1, p2, p3, p4, xnext = r
        df.loc[k] = {
            "t": t,
            "x0": x[0],
            "x1": x[1],
            "x2": x[2],
            "energy": e,
            "penalty": p,
            "p1": p1,
            "p2": p2,
            "p3": p3,
            "p4": p4,
        }
        print(
            "Time %f, x = %s, energy %f, penalty %f, penalty1 %f, penalty2 %f, penalty3 %f, penalty4 %f"
            % (t, x, e, p, p1, p2, p3, p4)
        )

    x = xnext
    print(f"Final: x = {x}")

    df.to_excel(export_to)

In [ ]:
# solve("output1.xlsx", x0=x0, dt=1e-4, T=1.0, simulator=runge_kutta_4)
import time  # import the time module

# Record the start time
start_time = time.time()

solve("output1.xlsx", x0=x0, dt=1e-4, T=2.0, simulator=runge_kutta_4)

# Record the end time
end_time = time.time()

# Calculate the total runtime
total_time = end_time - start_time
print(f"Total runtime: {total_time:.4f} seconds")


Time 0.000000, x = [0.4759 0.0892 0.5248], energy 1.029708, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000100, x = [0.4756 0.0895 0.5247], energy 1.027688, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000200, x = [0.4754 0.0899 0.5245], energy 1.025674, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000300, x = [0.4751 0.0902 0.5243], energy 1.023664, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000400, x = [0.4749 0.0905 0.5242], energy 1.021659, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000500, x = [0.4746 0.0908 0.524 ], energy 1.019659, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000600, x = [0.4744 0.0912 0.5238], energy 1.017664, penalty 0.000000, pena

Time 0.005900, x = [0.4614 0.1079 0.5149], energy 0.918482, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006000, x = [0.4612 0.1082 0.5147], energy 0.916729, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006100, x = [0.4609 0.1085 0.5146], energy 0.914981, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006200, x = [0.4607 0.1088 0.5144], energy 0.913236, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006300, x = [0.4605 0.1091 0.5143], energy 0.911496, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006400, x = [0.4602 0.1094 0.5141], energy 0.909760, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006500, x = [0.46   0.1097 0.5139], energy 0.908028, penalty 0.000000, pena

Time 0.011400, x = [0.4487 0.124  0.5062], energy 0.828048, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.011500, x = [0.4485 0.1243 0.5061], energy 0.826511, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.011600, x = [0.4483 0.1246 0.5059], energy 0.824978, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.011700, x = [0.448  0.1249 0.5058], energy 0.823449, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.011800, x = [0.4478 0.1251 0.5056], energy 0.821924, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.011900, x = [0.4476 0.1254 0.5055], energy 0.820402, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.012000, x = [0.4474 0.1257 0.5053], energy 0.818883, penalty 0.000000, pena

Time 0.017300, x = [0.4358 0.1401 0.4975], energy 0.743391, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017400, x = [0.4356 0.1403 0.4974], energy 0.742057, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017500, x = [0.4354 0.1406 0.4972], energy 0.740726, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017600, x = [0.4352 0.1408 0.4971], energy 0.739397, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017700, x = [0.435  0.1411 0.4969], energy 0.738072, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017800, x = [0.4348 0.1414 0.4968], energy 0.736750, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.017900, x = [0.4346 0.1416 0.4966], energy 0.735432, penalty 0.000000, pena

Time 0.023200, x = [0.4237 0.1549 0.4893], energy 0.669853, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023300, x = [0.4235 0.1551 0.4892], energy 0.668694, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023400, x = [0.4233 0.1554 0.4891], energy 0.667537, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023500, x = [0.4231 0.1556 0.4889], energy 0.666383, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023600, x = [0.4229 0.1558 0.4888], energy 0.665232, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023700, x = [0.4227 0.1561 0.4887], energy 0.664083, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.023800, x = [0.4225 0.1563 0.4885], energy 0.662937, penalty 0.000000, pena

...

Time 0.796900, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797000, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797100, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797200, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797300, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797400, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.797500, x = [0.2332 0.3537 0.413 ], energy 0.179882, penalty 0.000000, pena

Time 0.802600, x = [0.2332 0.3537 0.413 ], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.802700, x = [0.2332 0.3537 0.413 ], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.802800, x = [0.2332 0.3537 0.413 ], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.802900, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.803000, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.803100, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.803200, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, pena

Time 0.808400, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.808500, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.808600, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.808700, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.808800, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.808900, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.809000, x = [0.2332 0.3537 0.4131], energy 0.179881, penalty 0.000000, pena

Time 0.814200, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814300, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814400, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814500, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814600, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814700, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.814800, x = [0.2331 0.3537 0.4131], energy 0.179880, penalty 0.000000, pena

Time 0.820500, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.820600, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.820700, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.820800, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.820900, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.821000, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.821100, x = [0.2331 0.3537 0.4132], energy 0.179880, penalty 0.000000, pena